In [1]:
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
import holoviews as hv
hv.extension('bokeh')
from bokeh.plotting import show
warnings.filterwarnings('ignore')

In [ ]:
# Create an ExcelFile object
excel_file = pd.ExcelFile('US_Regional_Sales_Data.xls')
# Get the list of sheet names
sheet_names = excel_file.sheet_names
# Print the sheet names
print(sheet_names)

In [2]:
# Read Excel file into a Pandas DataFrame
Sales_df = pd.read_excel('US_Regional_Sales_Data.xls', sheet_name='Sales Orders Sheet')
Customers_df = pd.read_excel('US_Regional_Sales_Data.xls', sheet_name='Customers Sheet')
Stores_df = pd.read_excel('US_Regional_Sales_Data.xls', sheet_name='Store Locations Sheet')
Products_df = pd.read_excel('US_Regional_Sales_Data.xls', sheet_name='Products Sheet')
Regions_df = pd.read_excel('US_Regional_Sales_Data.xls', sheet_name='Regions Sheet')
Team_df = pd.read_excel('US_Regional_Sales_Data.xls', sheet_name='Sales Team Sheet')


In [3]:
Sales_df.columns

Index(['OrderNumber', 'Sales Channel', 'WarehouseCode', 'ProcuredDate',
       'OrderDate', 'ShipDate', 'DeliveryDate', 'CurrencyCode', '_SalesTeamID',
       '_CustomerID', '_StoreID', '_ProductID', 'Order Quantity',
       'Discount Applied', 'Unit Price', 'Unit Cost'],
      dtype='object')

In [4]:
Sales_df.shape

(7991, 16)

In [5]:
Sales_df['Recency'] = max(Sales_df['OrderDate'])-Sales_df['OrderDate']
Sales_df.sample(10)

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Price,Unit Cost,Recency
7860,SO - 0007961,In-Store,WARE-NBV1002,2020-09-26,2020-12-15,2021-01-03,2021-01-06,USD,7,6,63,17,7,0.075,1976.5,948.720,15 days
5030,SO - 0005131,Distributor,WARE-MKL1006,2019-12-01,2020-01-17,2020-02-10,2020-02-11,USD,25,31,334,24,6,0.075,3055.2,2230.296,348 days
3756,SO - 0003857,In-Store,WARE-NMK1003,2019-05-15,2019-08-16,2019-08-22,2019-08-27,USD,6,39,143,37,2,0.050,3852.5,3236.100,502 days
7002,SO - 0007103,In-Store,WARE-NMK1003,2020-06-18,2020-09-07,2020-09-29,2020-10-07,USD,7,17,96,6,8,0.100,3825.7,2869.275,114 days
537,SO - 000638,Online,WARE-NBV1002,2018-04-10,2018-08-03,2018-08-24,2018-08-26,USD,13,48,60,40,3,0.150,924.6,536.268,880 days
927,SO - 0001028,Online,WARE-NMK1003,2018-04-10,2018-09-15,2018-10-12,2018-10-21,USD,17,24,128,18,8,0.050,1118.9,637.773,837 days
7534,SO - 0007635,In-Store,WARE-UHY1004,2020-06-18,2020-11-06,2020-11-26,2020-12-05,USD,1,29,241,12,8,0.150,3872.6,2749.546,54 days
6275,SO - 0006376,Online,WARE-PUJ1005,2020-03-10,2020-06-12,2020-07-04,2020-07-13,USD,13,4,307,15,4,0.150,2425.4,970.160,201 days
4711,SO - 0004812,In-Store,WARE-NMK1003,2019-08-23,2019-12-13,2020-01-10,2020-01-18,USD,2,6,103,10,5,0.150,3845.8,1922.900,383 days
2925,SO - 0003026,Wholesale,WARE-NMK1003,2019-02-04,2019-05-09,2019-05-13,2019-05-16,USD,27,43,147,11,2,0.050,1949.7,1228.311,601 days


In [6]:
Sales_df = Sales_df[['_CustomerID', 'Sales Channel','Order Quantity',
       'Discount Applied', 'Unit Price', 'Unit Cost','Recency']]

In [7]:
Sales_df['Recency'] = Sales_df['Recency'].astype('string').str.replace('days', '')
Sales_df['Total Sales'] = Sales_df['Unit Price'] - (Sales_df['Unit Price']*Sales_df['Discount Applied'] - Sales_df['Unit Cost'])


In [8]:
Sales_df["Recency"]=Sales_df['Recency'].astype(int) 
Sales_df.dtypes

_CustomerID           int64
Sales Channel        object
Order Quantity        int64
Discount Applied    float64
Unit Price          float64
Unit Cost           float64
Recency               int32
Total Sales         float64
dtype: object

In [9]:
Sales_df.isnull().sum()

_CustomerID         0
Sales Channel       0
Order Quantity      0
Discount Applied    0
Unit Price          0
Unit Cost           0
Recency             0
Total Sales         0
dtype: int64

In [10]:
# Check duplicates
duplicates = Sales_df.duplicated(keep='first')
duplicate_rows = Sales_df[duplicates]
duplicate_rows

,_CustomerID,Sales Channel,Order Quantity,Discount Applied,Unit Price,Unit Cost,Recency,Total Sales


In [11]:
Sales_df.describe()

,_CustomerID,Order Quantity,Discount Applied,Unit Price,Unit Cost,Recency,Total Sales
count,7991.000000,7991.000000,7991.000000,7991.000000,7991.000000,7991.000000,7991.000000
mean,25.457014,4.525341,0.114394,2284.536504,1431.911054,471.540858,3453.042562
std,14.414883,2.312631,0.085570,1673.096364,1112.413043,273.860273,2565.338131
min,1.000000,1.000000,0.050000,167.500000,68.675000,0.000000,169.175000
25%,13.000000,3.000000,0.050000,1031.800000,606.115500,232.000000,1512.726000
50%,25.000000,5.000000,0.075000,1849.200000,1080.576000,472.000000,2788.272000
75%,38.000000,7.000000,0.150000,3611.300000,2040.250500,713.500000,5131.295500
max,50.000000,8.000000,0.400000,6566.000000,5498.556000,944.000000,11717.161000


In [12]:
Verify_Sales = Sales_df['Unit Cost']>Sales_df['Unit Price']
print(Verify_Sales)
Verify_values = ((Sales_df['Discount Applied']>1 )|(Sales_df['Order Quantity']<0) |(Sales_df['Unit Price'] <0) | (Sales_df['Unit Cost']<0))
print(Verify_values)

0       False
1       False
2       False
3       False
4       False
        ...  
7986    False
7987    False
7988    False
7989    False
7990    False
Length: 7991, dtype: bool
0       False
1       False
2       False
3       False
4       False
        ...  
7986    False
7987    False
7988    False
7989    False
7990    False
Length: 7991, dtype: bool


In [13]:
# there are outliers but lets proceed first before removing outliers
df = Sales_df.groupby('_CustomerID').agg({
    'Order Quantity' :'sum',
    'Discount Applied': 'mean', 
    'Unit Price': 'sum', 
    'Unit Cost' : 'sum',
    'Recency' : 'mean',
    'Total Sales' : 'sum',
    'Sales Channel': lambda x: x.mode().iloc[0]
}).reset_index()